# Simplified Agent Creation with `FlotorchADKAgent`

This notebook demonstrates the streamlined approach to creating agents using the `FlotorchADKAgent` client. This powerful feature allows you to manage your agent's core configuration (its name, instructions, and persona) on the Flotorch platform UI and then load it directly into your application code.

### Prerequesit
Configure agent, API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Advantage:

The primary benefit is the **decoupling of configuration from code**. You can update your agent's behavior, instructions, or model choice through a user-friendly interface without ever needing to change and redeploy your application code. This notebook focuses on this core capability in its simplest form.

## 1. Environment Setup and Imports

The following cells handle the complete setup of the environment. This includes installing the necessary packages, defining your API credentials, and importing the required components. The key import for this workflow is `FlotorchADKAgent`.

In [ ]:
# install flotorch adk package
%pip install --pre flotorch[adk]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
AGENT_NAME = "<flotorch agent name>"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"

In [ ]:
# import necessary libraries
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.sessions import FlotorchADKSession
from google.adk import Runner
from google.genai import types

print("Imported necessary libraries successfully")

## 2. Initializing the Remote Agent

This is the core of the simplified workflow. We create an instance of the `FlotorchADKAgent` client, providing only the `agent_name` and API credentials. The client connects to the Flotorch platform and fetches the complete agent configuration that you have defined in the UI. No instructions, tools, or persona details are needed in the code.

In [ ]:
flotorch_client =  FlotorchADKAgent(
        agent_name=AGENT_NAME,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL
    )

# Initialize the Flotorch agent
agent = flotorch_client.get_agent()
print(f"FlotorchADKAgent '{agent.name}' created successfully.")

## 3. Configuring the Runner

Next, we configure the `Runner`. This component orchestrates the interaction between the user and the agent. We provide it with our remotely-fetched `agent` and a `FlotorchADKSession` service to handle the short-term memory for the conversation.

In [ ]:
# Initialize Session
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)
    
# Initialize runner
runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service
    )

print("Runner configured successfully for FlotorchADKAgent.")

## 4. Interactive Chat

With the setup complete, you can now interact with your agent. The agent's responses and behavior are governed by the instructions and settings you defined on the Flotorch platform UI, not by any code written in this notebook.

In [ ]:
def run_single_turn(query, session_id, user_id, runner):
    """
    Send a message to the FlotorchADKAgent and get the response.
    """
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that request."

async def chat_with_agent(query,session_id):
    """
    Chat function that takes a user query and returns the FlotorchADKAgent's response.
    """
    response = run_single_turn(query, session_id, USER_ID, runner)
    return response

print("Chat function ready. Use chat_with_agent('your query') to interact with the FlotorchADKAgent.")

In [ ]:
# Example: Basic interaction
session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
response = await chat_with_agent("Hello, I am John",session.id)
print(f"Query: Hello, I am John")
print(f"Response: {response}")

## Summary

This notebook demonstrated the power and simplicity of using `FlotorchADKAgent` to drive AI applications. By centralizing the agent's configuration on the Flotorch platform, we created a more flexible, maintainable, and scalable system.

### Key Achievements:

- **Simplified Codebase**: We successfully instantiated a fully functional agent with minimal code, as its core logic (persona, instructions) resides on the Flotorch platform.
- **Decoupled Architecture**: This workflow effectively separates the agent's behavioral configuration from the application's operational code, allowing different teams to manage each component independently.
- **Agile Development**: Agent behavior can be tweaked, tested, and updated through the Flotorch UI without requiring any code changes or service redeployments, accelerating the development lifecycle.